###Fuente de la data

Los micro-datos gestionados en este notebook son tomados de www.datos.gov.co y corresponden a datos cuya fuente primaria es el Departamento Administrativo para la Prosperidad Social

In [1]:
import pandas as pd

###Descarga del dataset desde el endpoint - archivo xfif-myr2.csv

Contiene datos anonimizados del programa Más Familias en Acción, determinando características generales de los beneficiarios, valor y tipo de incentivo recibido, con corte a 31 de julio de 2024


In [3]:
familias_en_accion_prosperidad_social = pd.read_csv("https://www.datos.gov.co/resource/xfif-myr2.csv?$limit=5000000", dtype={'codigomunicipioatencion': str})
familias_en_accion_prosperidad_social.head(3)

,bancarizado,codigodepartamentoatencion,codigomunicipioatencion,discapacidad,estadobeneficiario,etnia,fechainscripcionbeneficiario,genero,nivelescolaridad,nombredepartamentoatencion,...,tipoasignacionbeneficio,tipobeneficio,tipodocumento,tipopoblacion,rangobeneficioconsolidadoasignado,rangoultimobeneficioasignado,fechaultimobeneficioasignado,rangoedad,titular,cantidaddebeneficiarios
0,SI,8,08421,NO,ACTIVO,AFROCOLOMBIANO – NEGRO,2012-12-01,Hombre,ND,ATLANTICO,...,MONETARIO,ND,CC,UNIDOS,4.500.001 - 6.000.000,0 - 1.300.000,2018-01-01,30-49,SI,1
1,NO,13,13673,NO,NO ACTIVO,ND,2012-11-01,Mujer,ND,BOLIVAR,...,ND,ND,TI,ND,0 - 1.500.000,0 - 1.300.000,1900-01-01,18-29,NO,21
2,SI,8,08421,NO,ACTIVO,AFROCOLOMBIANO – NEGRO,2012-12-01,Hombre,ND,ATLANTICO,...,MONETARIO,ND,CC,UNIDOS,4.500.001 - 6.000.000,0 - 1.300.000,2018-01-01,30-49,SI,1


###Revisión y limpieza para integrarlo a la base de datos

### - Resumen de la estructura del dataset

In [4]:
familias_en_accion_prosperidad_social.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3958594 entries, 0 to 3958593
Data columns (total 22 columns):
 #   Column                             Dtype 
---  ------                             ----- 
 0   bancarizado                        object
 1   codigodepartamentoatencion         int64 
 2   codigomunicipioatencion            object
 3   discapacidad                       object
 4   estadobeneficiario                 object
 5   etnia                              object
 6   fechainscripcionbeneficiario       object
 7   genero                             object
 8   nivelescolaridad                   object
 9   nombredepartamentoatencion         object
 10  nombremunicipioatencion            object
 11  pais                               object
 12  tipoasignacionbeneficio            object
 13  tipobeneficio                      object
 14  tipodocumento                      object
 15  tipopoblacion                      object
 16  rangobeneficioconsolidadoasignado  o

- Descripcion de cada columna

In [5]:
column_mapping = {
    "bancarizado": "El participante se encuentra en bancarizado",
    "codigodepartamentoatencion": "Código DANE del departamento en el cual el beneficiario recibe atención",
    "codigomunicipioatencion": "Código DANE del municipio en el cual el beneficiario recibe atención",
    "discapacidad": "El beneficio posee alguna discapacidad o limitación",
    "estadobeneficiario": "Estado en el cual se encuentra actualmente el beneficiario del programa",
    "etnia": "Étnia a la cual pertenece el beneficiario",
    "fechainscripcionbeneficiario": "Fecha de inscripción al programa. La información no disponible se marca como 01/01/1900.",
    "genero": "Género del participante del programa",
    "nivelescolaridad": "Nivel de escolaridad beneficiario del programa",
    "nombredepartamentoatencion": "Nombre del departamento en el cual recibe atención",
    "nombremunicipioatencion": "Nombre del municipio en el cual recibe atención",
    "pais": "País en el cual reside el beneficiario",
    "tipoasignacionbeneficio": "Tipo de beneficio en lugar o sitio",
    "tipobeneficio": "Tipo de beneficio recibido",
    "tipodocumento": "Tipo de documento del beneficiario del programa",
    "tipopoblacion": "Tipo de población atendida",
    "rangobeneficioconsolidadoasignado": "Rango en el cual se encuentra el valor del beneficio recibido hasta el momento",
    "rangoultimobeneficioasignado": "Rango en el cual se encuentra el valor del último beneficio recibido",
    "fechaultimobeneficioasignado": "Fecha en la cual el beneficiario recibió el último incentivo. La información no disponible se marca como 01/01/1900.",
    "rangoedad": "Rango de edades en la cual se encuentra el beneficiario",
    "titular": "El beneficiario es titular o no del programa",
    "cantidaddebeneficiarios": "Indica la cantidad de beneficiarios que cumplen con las características."
}

In [6]:
columns = familias_en_accion_prosperidad_social.columns

In [7]:
# Verificar el numero de valores únicos que se guarda en cada columna
for col in columns:
  print(f'{col}: {familias_en_accion_prosperidad_social[col].nunique()}')

bancarizado: 3
codigodepartamentoatencion: 33
codigomunicipioatencion: 1110
discapacidad: 3
estadobeneficiario: 2
etnia: 11
fechainscripcionbeneficiario: 75
genero: 3
nivelescolaridad: 9
nombredepartamentoatencion: 33
nombremunicipioatencion: 1026
pais: 7
tipoasignacionbeneficio: 2
tipobeneficio: 10
tipodocumento: 6
tipopoblacion: 6
rangobeneficioconsolidadoasignado: 5
rangoultimobeneficioasignado: 3
fechaultimobeneficioasignado: 7
rangoedad: 5
titular: 2
cantidaddebeneficiarios: 1228


In [18]:
# Eliminar columnas que se considera innecesarias para el proyecto
delete_columns = ['bancarizado','codigodepartamentoatencion', 'discapacidad', 'pais', 'tipodocumento', 'fechaultimobeneficioasignado', 'titular'  ]
# Eliminando las columnas especificadas
familias_en_accion_prosperidad_social = familias_en_accion_prosperidad_social.drop(delete_columns, axis=1)

### Estandarización de categorizaciones

La estandarización de categorizaciones es el proceso de uniformizar y normalizar los valores de las categorías en un conjunto de datos para asegurar la consistencia y evitar discrepancias. Esto es crucial para la calidad y precisión de los análisis

In [19]:
# Obtener las columnas que no son numéricas
non_numeric_columns = familias_en_accion_prosperidad_social.select_dtypes(exclude=['int64', 'float64']).columns
print(non_numeric_columns)

Index(['codigomunicipioatencion', 'estadobeneficiario', 'etnia',
       'fechainscripcionbeneficiario', 'genero', 'nivelescolaridad',
       'nombredepartamentoatencion', 'nombremunicipioatencion',
       'tipoasignacionbeneficio', 'tipobeneficio', 'tipopoblacion',
       'rangobeneficioconsolidadoasignado', 'rangoultimobeneficioasignado',
       'rangoedad'],
      dtype='object')


- Borrar espacios en blanco al principio y al final, cambiar a mayúsculas, remover acentos y eliminar signos extraños

In [22]:
import unicodedata

def remove_accents_and_special_chars(input_str):
    # Normalizar la cadena a NFKD
    nfkd_form = unicodedata.normalize('NFKD', input_str)

    # Eliminar acentos
    no_accents = ''.join([c for c in nfkd_form if not unicodedata.combining(c)])

    # Definir caracteres no deseados
    unwanted_chars =  [',', ';', '!', '?', '#', '$', '%', '"', "'", '/', '\\', '|', '-']

    # Eliminar caracteres no deseados
    cleaned_str = ''.join([c for c in no_accents if c not in unwanted_chars])

    # Remover espacios en blanco al principio y al final, y convertir a mayúsculas
    result = cleaned_str.strip().upper()

    return result

In [23]:
# Aplicar la función a todas las columnas categóricas
familias_en_accion_prosperidad_social[non_numeric_columns] = familias_en_accion_prosperidad_social[non_numeric_columns].astype(str)
for col in non_numeric_columns:
    familias_en_accion_prosperidad_social[col] = familias_en_accion_prosperidad_social[col].apply(remove_accents_and_special_chars)

In [24]:
# Imprimir categorías únicas para columnas de tipo object
for column in non_numeric_columns:
    print(f"Categorías en la columna '{column}':")
    print(familias_en_accion_prosperidad_social[column].unique())
    print()

Categorías en la columna 'codigomunicipioatencion':
['08421' '13673' '41770' ... '91669' '27086' '97889']

Categorías en la columna 'estadobeneficiario':
['ACTIVO' 'NO ACTIVO']

Categorías en la columna 'etnia':
['AFROCOLOMBIANO – NEGRO' 'ND' 'INDIGENA' 'RAIZAL' 'AFROCOLOMBIANO  NEGRO'
 'MESTIZO' 'PALENQUERO' 'ROM O GITANO' 'ROM' 'AFROCOLOMBIANO'
 'SIN ESPECIFICAR']

Categorías en la columna 'fechainscripcionbeneficiario':
['20121201' '20121101' '20130201' '20140501' '20121001' '20150401'
 '20130101' '20160601' '20180501' '20150601' '20170601' '20130601'
 '20161201' '20160701' '20131001' '20131201' '20161101' '20130401'
 '20150901' '20140801' '20131101' '20130701' '20171001' '20151001'
 '20150501' '20150801' '20130901' '20150301' '20141001' '20150101'
 '20140701' '20181101' '20151101' '20160901' '20140301' '20170301'
 '20141101' '20161001' '20160401' '20160201' '20180201' '20180301'
 '20140901' '20141201' '20150201' '20140401' '20170101' '20160801'
 '20150701' '20160301' '20181001' '20

- Estandarizar algunas columnas categoricas

In [25]:
# Columna etnia
# Crear una función para estandarizar la columna 'etnia'
def estandarizar_etnia(etnia):
    etnia = etnia.strip().upper()  # Eliminar espacios y poner en mayúsculas
    mapping = {
        'AFROCOLOMBIANO – NEGRO': 'AFROCOLOMBIANO NEGRO',
        'AFROCOLOMBIANO  NEGRO': 'AFROCOLOMBIANO NEGRO',
        'AFROCOLOMBIANO': 'AFROCOLOMBIANO',
        'INDIGENA': 'INDIGENA',
        'RAIZAL': 'RAIZAL',
        'MESTIZO': 'MESTIZO',
        'PALENQUERO': 'PALENQUERO',
        'ROM O GITANO': 'ROM',
        'ROM': 'ROM',
        'ND': 'SIN ESPECIFICAR',
        'SIN ESPECIFICAR': 'SIN ESPECIFICAR'
    }
    return mapping.get(etnia, etnia)  # Devolver el valor estandarizado

# Aplicar la función para estandarizar la columna 'etnia'
familias_en_accion_prosperidad_social['etnia'] = familias_en_accion_prosperidad_social['etnia'].apply(estandarizar_etnia)

# Ver los resultados estandarizados
print(familias_en_accion_prosperidad_social['etnia'].unique())

['AFROCOLOMBIANO NEGRO' 'SIN ESPECIFICAR' 'INDIGENA' 'RAIZAL' 'MESTIZO'
 'PALENQUERO' 'ROM' 'AFROCOLOMBIANO']


In [26]:
# columna genero
# Crear una función para estandarizar la columna 'genero'
def estandarizar_genero(genero):
    if genero == 'HOMBRE':
        return 'HOMBRE'
    elif genero == 'MUJER':
        return 'MUJER'
    elif genero == 'ND':
        return 'SIN ESPECIFICAR'  # O 'NO DISPONIBLE', según tu preferencia
    else:
        return 'SIN ESPECIFICAR'  # Para cualquier valor inesperado

# Aplicar la función para estandarizar la columna 'genero'
familias_en_accion_prosperidad_social['genero'] = familias_en_accion_prosperidad_social['genero'].apply(estandarizar_genero)

# Ver los resultados estandarizados
print(familias_en_accion_prosperidad_social['genero'].unique())

['HOMBRE' 'MUJER' 'SIN ESPECIFICAR']


In [28]:
# Crear una función para estandarizar la columna 'nivelescolaridad'
def estandarizar_nivel_escolaridad(nivel):
    nivel = nivel.strip().upper()  # Eliminar espacios y poner en mayúsculas
    mapping = {
        'ND': 'SIN ESPECIFICAR',
        'PRIMARIA': 'PRIMARIA',
        'TRANSICION': 'TRANSICION',
        'SECUNDARIA': 'SECUNDARIA',
        'SIN ESPECIFICAR': 'SIN ESPECIFICAR',
        'POSGRADO': 'POSGRADO',
        'TECNOLOGO': 'TECNOLOGO',
        'TECNICO': 'TECNICO',
        'PREGRADO': 'PREGRADO'
    }
    return mapping.get(nivel, nivel)  # Devolver el valor estandarizado

# Aplicar la función para estandarizar la columna 'nivelescolaridad'
familias_en_accion_prosperidad_social['nivelescolaridad'] = familias_en_accion_prosperidad_social['nivelescolaridad'].apply(estandarizar_nivel_escolaridad)

# Ver los resultados estandarizados
print(familias_en_accion_prosperidad_social['nivelescolaridad'].unique())

['SIN ESPECIFICAR' 'PRIMARIA' 'TRANSICION' 'SECUNDARIA' 'POSGRADO'
 'TECNOLOGO' 'TECNICO' 'PREGRADO']


In [30]:
def estandarizar_tipo_asignacion(beneficio):
    mapping = {
        'MONETARIO': 'MONETARIO',
        'ND': 'SIN ASIGNACIÓN'  # O 'NO DISPONIBLE', según tu preferencia
    }
    return mapping.get(beneficio, beneficio)  # Devolver el valor estandarizado

# Aplicar la función para estandarizar la columna 'tipoasignacionbeneficio'
familias_en_accion_prosperidad_social['tipoasignacionbeneficio'] = (
    familias_en_accion_prosperidad_social['tipoasignacionbeneficio']
    .apply(estandarizar_tipo_asignacion)
)

# Ver los resultados estandarizados
print(familias_en_accion_prosperidad_social['tipoasignacionbeneficio'].unique())

['MONETARIO' 'SIN ASIGNACIÓN']


In [31]:
# Crear una función para estandarizar la columna 'tipobeneficio'
def estandarizar_tipobeneficio(tipobeneficio):
    tipobeneficio = tipobeneficio.strip().upper()  # Eliminar espacios y poner en mayúsculas
    mapping = {
        'ND': 'SIN ESPECIFICAR',
        'EDUCACION PRIMARIANUTRICION MENOR': 'EDUCACION PRIMARIA NUTRICION MENOR',
        'NUTRICION MENOR': 'NUTRICION MENOR',
        'TRANSICION': 'TRANSICION',
        'TRANSICIONNUTRICION MENOR': 'TRANSICION NUTRICION MENOR',
        'EDUCACION PRIMARIA': 'EDUCACION PRIMARIA',
        'EDUCACION SECUNDARIA': 'EDUCACION SECUNDARIA',
        'NUTRICION': 'NUTRICION',
        'EDUCACION SECUNDARIANUTRICION': 'EDUCACION SECUNDARIA NUTRICION',
        'EDUCACION PRIMARIANUTRICION': 'EDUCACION PRIMARIA NUTRICION'
    }
    return mapping.get(tipobeneficio, tipobeneficio)  # Devolver el valor estandarizado

# Aplicar la función para estandarizar la columna 'tipobeneficio'
familias_en_accion_prosperidad_social['tipobeneficio'] = familias_en_accion_prosperidad_social['tipobeneficio'].apply(estandarizar_tipobeneficio)

# Ver los resultados estandarizados
print(familias_en_accion_prosperidad_social['tipobeneficio'].unique())

['SIN ESPECIFICAR' 'EDUCACION PRIMARIA NUTRICION MENOR' 'NUTRICION MENOR'
 'TRANSICION' 'TRANSICION NUTRICION MENOR' 'EDUCACION PRIMARIA'
 'EDUCACION SECUNDARIA' 'NUTRICION' 'EDUCACION SECUNDARIA NUTRICION'
 'EDUCACION PRIMARIA NUTRICION']


In [32]:
# Crear una función para estandarizar la columna 'tipopoblacion'
def estandarizar_tipopoblacion(tipopoblacion):
    tipopoblacion = tipopoblacion.strip().upper()  # Eliminar espacios y poner en mayúsculas
    mapping = {
        'UNIDOS': 'UNIDOS',
        'ND': 'NO DISPONIBLE',
        'SISBEN': 'SISBEN',
        'INDIGENAS': 'INDÍGENAS',
        'DESPLAZADOS': 'DESPLAZADOS',
        'TRANSICION': 'TRANSICIÓN'
    }
    return mapping.get(tipopoblacion, tipopoblacion)  # Devolver el valor estandarizado

# Aplicar la función para estandarizar la columna 'tipopoblacion'
familias_en_accion_prosperidad_social['tipopoblacion'] = familias_en_accion_prosperidad_social['tipopoblacion'].apply(estandarizar_tipopoblacion)

# Ver los resultados estandarizados
print(familias_en_accion_prosperidad_social['tipopoblacion'].unique())

['UNIDOS' 'NO DISPONIBLE' 'SISBEN' 'INDÍGENAS' 'DESPLAZADOS' 'TRANSICIÓN']


In [33]:
# Crear una función para convertir los rangos a categorías de edad
def categorizar_edad(rango):
    if rango == '0617':
        return 'NINOS Y ADOLESCENTES'  # 0-17 años
    elif rango == '1829':
        return 'JOVENES ADULTOS'        # 18-29 años
    elif rango == '3049':
        return 'ADULTOS DE MEDIANA EDAD'  # 30-49 años
    elif rango == '5065':
        return 'ADULTOS MAYORES'        # 50-64 años
    elif rango == '>65':
        return 'ANCIANOS'               # 65 años y más
    else:
        return 'SIN ESPECIFICAR'  # Para cualquier otro caso no previsto

# Aplicar la función para categorizar la columna 'rangoedad'
familias_en_accion_prosperidad_social['rangoedad'] = familias_en_accion_prosperidad_social['rangoedad'].apply(categorizar_edad)

# Ver los resultados categorizados
print(familias_en_accion_prosperidad_social['rangoedad'].unique())

['ADULTOS DE MEDIANA EDAD' 'JOVENES ADULTOS' 'NINOS Y ADOLESCENTES'
 'ADULTOS MAYORES' 'ANCIANOS']


### Hacer coincidir los códigos de los municipios con el formato de los códigos guardados en la base de datos

Los códigos reales de los municipios de Colombia, están almacenados en la base de datos PostgreSQL del proyecto, en la tabla municipalities dentro del campo dept_mpio_code, junto con la informacion necesaria para georeferenciar todos los municipios y departamentos de Colombia. Este campo guarda el código del municipio en un formato string de exactamente 5 caracteres, los dos primeros corresponden al departamento y los tres restantes al municipio.


Para hacer esta verificación, previamente exportamos desde la base de datos PostgreSQL un DataFrame con los siguientes campos: dept_name, mpio_name y dept_mpio_code, los cuales contienen la información de los departamentos y municipios oficiales, junto con sus respectivos códigos. Este DataFrame se carga en la siguiente celda y se utiliza para comparar con la columna "codigomunicipioatencion"

In [34]:
dept_mpios_codes = pd.read_csv("/content/drive/MyDrive/analytics_data_proyect/deptos_mupios.csv", index_col=0, dtype={'dept_mpio_code': str})
print(dept_mpios_codes.info())
dept_mpios_codes.head()

<class 'pandas.core.frame.DataFrame'>
Index: 1121 entries, 0 to 1120
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   dept_mpio_code  1121 non-null   object
 1   dept_name       1121 non-null   object
 2   mupio_name      1121 non-null   object
dtypes: object(3)
memory usage: 35.0+ KB
None


,dept_mpio_code,dept_name,mupio_name
0,97001,VAUPES,MITU
1,97161,VAUPES,CARURU
2,97511,VAUPES,PACOA
3,97666,VAUPES,TARAIRA
4,97777,VAUPES,PAPUNAHUA


- Longitud de los códigos municipios en el df

In [37]:
# Convertir a strings
familias_en_accion_prosperidad_social["codigomunicipioatencion"] = familias_en_accion_prosperidad_social["codigomunicipioatencion"].astype(str)

# Calcular la longitud de cada valor en la columna
longitud_mpio = familias_en_accion_prosperidad_social["codigomunicipioatencion"].apply(len)

# Longitudes de los códigos
print(f'Los códigos de los mpios tiene longitudes de :{longitud_mpio.unique()} caracteres')

Los códigos de los mpios tiene longitudes de :[5] caracteres


In [38]:
# Contar registros por longitud
long_df = longitud_mpio.value_counts().reset_index()
long_df.rename(columns={'cod_mpio': 'no_dígitos_codigo_dane'}, inplace=True)
long_df['percentage'] = (long_df['count'] / len(longitud_mpio))
long_df.head()

,codigomunicipioatencion,count,percentage
0,5,3958594,1.0


In [40]:
# Mostrar una muestra de registros para cada longitud
for longitud in longitud_mpio.value_counts().index:
    print(f"Muestra de registros con longitud {longitud}:")
    muestra = familias_en_accion_prosperidad_social[longitud_mpio == longitud].head(5)  # Muestra de los primeros 5 registros
    print(muestra[['codigomunicipioatencion']])
    print()

Muestra de registros con longitud 5:
  codigomunicipioatencion
0                   08421
1                   13673
2                   08421
3                   08421
4                   08421



Nota: Todos los códigos del df que se esta analizando tiene un string de 5 digitos igual al almacenado en la base de datos, asi que pasamos  a verificar si todos los códigos de este df corresponden a valores reales

- Verificar que los códigos de municipios que quedaron en el dataset correspondan solamente a códigos reales almacenados en la base de datos

In [41]:
# Función para comparar listas y mostrar diferencias
def compare_lists(df1_col, df2_col, label1, label2):
    # Extraer listas únicas y normalizar
    list1 = set(df1_col.str.strip().str.upper().unique())
    list2 = set(df2_col.str.strip().str.upper().unique())

    # Encontrar diferencias
    only_in_list1 = list1 - list2
    only_in_list2 = list2 - list1

    # Imprimir resultados
    print(f"{label1} que no están en {label2}:")
    print(only_in_list1)

In [42]:
# Comparar listas de códigos
compare_lists(familias_en_accion_prosperidad_social['codigomunicipioatencion'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en familias_en_accion_prosperidad_social", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en familias_en_accion_prosperidad_social que no están en Códigos de municipios dept_mpios_codes:
{'27086'}


- Analizar cod de mpio que no corresponde a un cod real

In [47]:
mpio_cod_27086 = familias_en_accion_prosperidad_social[familias_en_accion_prosperidad_social['codigomunicipioatencion']=='27086']
mpio_cod_27086['nombremunicipioatencion'].value_counts()

,count
nombremunicipioatencion,
BELEN DE BAJIRA,1


Nota: BELEN DE BIJARA figura en la base de datos como un corregimiento del municipio de MUTATA.

In [49]:
familias_en_accion_prosperidad_social[familias_en_accion_prosperidad_social['nombremunicipioatencion']== 'MUTATA'].head(1)

,codigomunicipioatencion,estadobeneficiario,etnia,fechainscripcionbeneficiario,genero,nivelescolaridad,nombredepartamentoatencion,nombremunicipioatencion,tipoasignacionbeneficio,tipobeneficio,tipopoblacion,rangobeneficioconsolidadoasignado,rangoultimobeneficioasignado,rangoedad,cantidaddebeneficiarios
48,05480,ACTIVO,SIN ESPECIFICAR,20161101,MUJER,PRIMARIA,ANTIOQUIA,MUTATA,SIN ASIGNACIÓN,EDUCACION PRIMARIA,DESPLAZADOS,0 1.500.000,0 1.300.000,NINOS Y ADOLESCENTES,2


In [50]:
# Reemplazar el valor '27086' por '05480' en la columna 'codigomunicipioatencion'
familias_en_accion_prosperidad_social['codigomunicipioatencion'] = familias_en_accion_prosperidad_social['codigomunicipioatencion'].replace('27086', '05480')

- Verificar si cambio se aplicó

In [51]:
# Comparar listas de códigos
compare_lists(familias_en_accion_prosperidad_social['codigomunicipioatencion'], dept_mpios_codes['dept_mpio_code'],
              "Códigos de municipios en familias_en_accion_prosperidad_social", "Códigos de municipios dept_mpios_codes")

Códigos de municipios en familias_en_accion_prosperidad_social que no están en Códigos de municipios dept_mpios_codes:
set()


- Convertir columna fechainscripcionbeneficiario a tipo date

In [54]:
# Hacer una copia explícita del DataFrame
#familias_en_accion_prosperidad_social = familias_en_accion_prosperidad_social.copy()

# Convertir la columna 'fecha_hecho' a tipo datetime (sin especificar formato)
familias_en_accion_prosperidad_social['fechainscripcionbeneficiario'] = pd.to_datetime(familias_en_accion_prosperidad_social['fechainscripcionbeneficiario'], errors='coerce')

# Extraer solo el año
familias_en_accion_prosperidad_social['fechainscripcionbeneficiario'] = familias_en_accion_prosperidad_social['fechainscripcionbeneficiario'].dt.year

In [55]:
familias_en_accion_prosperidad_social['fechainscripcionbeneficiario'].unique()

array([2012, 2013, 2014, 2015, 2016, 2018, 2017], dtype=int32)

### Procesamiento final como preparación para integrarlo a la bd de datos del proyecto

- Eliminar columnas que dejaron de ser necesarias

In [58]:
# Eliminar columnas 'departamento', 'municipio' que ya no son necesarias
columns_to_drop = ['nombredepartamentoatencion', 'nombremunicipioatencion']
familias_en_accion_prosperidad_social = familias_en_accion_prosperidad_social.drop(columns=columns_to_drop)

In [56]:
# Adicionar columna para trazabilidad de la fuente
# Lista de tuplas
familias_en_accion_prosperidad_social['source_id'] = 116

In [59]:
familias_en_accion_prosperidad_social.columns

Index(['codigomunicipioatencion', 'estadobeneficiario', 'etnia',
       'fechainscripcionbeneficiario', 'genero', 'nivelescolaridad',
       'tipoasignacionbeneficio', 'tipobeneficio', 'tipopoblacion',
       'rangobeneficioconsolidadoasignado', 'rangoultimobeneficioasignado',
       'rangoedad', 'cantidaddebeneficiarios', 'source_id'],
      dtype='object')

In [60]:
# Ajustar nombre de columnas

# Definir el diccionario de traducción
translation_map = {
    'fechainscripcionbeneficiario':'year',
    'codigomunicipioatencion': 'dane_code',
    'estadobeneficiario': 'condition',
    'etnia': 'ethnic_group',
    'genero': 'gender',
    'nivelescolaridad': 'educational_level',
    'tipoasignacionbeneficio': 'type_of_benefit_assignment',
    'tipobeneficio': 'type_of_benefit',
    'tipopoblacion': 'type_of_population',
    'rangobeneficioconsolidadoasignado': 'consolidated_benefit_range_assigned',
    'rangoultimobeneficioasignado': 'last_beneficit_range',
    'rangoedad': 'age',
    'cantidaddebeneficiarios': 'number_of_beneficiaries',
    'source_id': 'source_id'
}

# Renombrar las columnas
familias_en_accion_prosperidad_social.rename(columns=translation_map, inplace=True)

In [61]:
#Estructura final del dataset a integrar a la base de datos
familias_en_accion_prosperidad_social.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3958594 entries, 0 to 3958593
Data columns (total 14 columns):
 #   Column                               Dtype 
---  ------                               ----- 
 0   dane_code                            object
 1   condition                            object
 2   ethnic_group                         object
 3   year                                 int32 
 4   gender                               object
 5   educational_level                    object
 6   type_of_benefit_assignment           object
 7   type_of_benefit                      object
 8   type_of_population                   object
 9   consolidated_benefit_range_assigned  object
 10  last_beneficit_range                 object
 11  age                                  object
 12  number_of_beneficiaries              int64 
 13  source_id                            int64 
dtypes: int32(1), int64(2), object(11)
memory usage: 407.7+ MB


In [62]:
familias_en_accion_prosperidad_social.head()

,dane_code,condition,ethnic_group,year,gender,educational_level,type_of_benefit_assignment,type_of_benefit,type_of_population,consolidated_benefit_range_assigned,last_beneficit_range,age,number_of_beneficiaries,source_id
0,08421,ACTIVO,AFROCOLOMBIANO NEGRO,2012,HOMBRE,SIN ESPECIFICAR,MONETARIO,SIN ESPECIFICAR,UNIDOS,4.500.001 6.000.000,0 1.300.000,ADULTOS DE MEDIANA EDAD,1,116
1,13673,NO ACTIVO,SIN ESPECIFICAR,2012,MUJER,SIN ESPECIFICAR,SIN ASIGNACIÓN,SIN ESPECIFICAR,NO DISPONIBLE,0 1.500.000,0 1.300.000,JOVENES ADULTOS,21,116
2,08421,ACTIVO,AFROCOLOMBIANO NEGRO,2012,HOMBRE,SIN ESPECIFICAR,MONETARIO,SIN ESPECIFICAR,UNIDOS,4.500.001 6.000.000,0 1.300.000,ADULTOS DE MEDIANA EDAD,1,116
3,08421,ACTIVO,AFROCOLOMBIANO NEGRO,2012,HOMBRE,SIN ESPECIFICAR,MONETARIO,SIN ESPECIFICAR,UNIDOS,4.500.001 6.000.000,0 1.300.000,ADULTOS DE MEDIANA EDAD,1,116
4,08421,ACTIVO,AFROCOLOMBIANO NEGRO,2012,HOMBRE,SIN ESPECIFICAR,MONETARIO,SIN ESPECIFICAR,UNIDOS,4.500.001 6.000.000,0 1.300.000,ADULTOS DE MEDIANA EDAD,1,116


## Salvar en archivo csv en el drive

In [63]:
# Guardar en archivos CSV en el drive
familias_en_accion_prosperidad_social.to_csv('/content/drive/MyDrive/analytics_data_proyect/initial_transformation/familias_en_accion_prosperidad_social.csv', index=False)